In [1]:
!pip install --user uproot
import sys
sys.path.append("/home/acraplet/Alie/UROP2021/AnalyserWork/RootFiles")
import uproot 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
import xgboost as xgb
import matplotlib as mpl
#mpl.use('Agg')
mpl.use('TkAgg')
import matplotlib.pyplot as plt

In [2]:
# loading the trees
#tree = uproot.open("/home/acraplet/Alie/UROP2021/AnalyserWork/RootFiles/WminusHToTauTauUncorrelatedDecay_Filtered_tt_2018_BDTOddEven_test.root")["ntuple"]#RootFiles/WplusHToTauTauUncorrelatedDecay_Filtered_tt_2018_withBDT.root")["ntuple"]
# tree = uproot.open("/home/acraplet/Alie/UROP2021/AnalyserWork/Test2.root")["ntuple"]
tree = uproot.open("/home/acraplet/Alie/UROP2021/AnalyserWork/RootFiles/GluGluHToTauTauUncorrelatedDecay_Filtered_tt_2018_BDTOddEven.root")["ntuple"]
tree2 = uproot.open("/home/acraplet/Alie/UROP2021/AnalyserWork/RootFiles/VBFHToTauTauUncorrelatedDecay_Filtered_tt_2018.root")["ntuple"]

In [4]:
# define what variables are to be read into the dataframe

variables = [  "wt_cp_sm", "wt_cp_ps", "wt_cp_mm",
                "rand",
                "pt_1","pt_2",
                "met",
                "aco_angle_1", "aco_angle_5", "aco_angle_6",
                "y_1_1", "y_1_2",
                "ip_sig_1", "ip_sig_2",
                #"BDTOddEven",
                "mva_dm_1","mva_dm_2",
                "tau_decay_mode_1","tau_decay_mode_2",
                "deepTauVsJets_medium_1","deepTauVsJets_medium_2",
                "deepTauVsEle_vvloose_1","deepTauVsEle_vvloose_2",
                "deepTauVsMu_vloose_1","deepTauVsMu_vloose_2",
                "trg_doubletau", "pv_angle_new", #"BDTOddEven"
             ]


pi_1 = ["pi_E_1", "pi_px_1", "pi_py_1", "pi_pz_1"]
pi_2 = ["pi_E_2", "pi_px_2", "pi_py_2", "pi_pz_2"]

pi0_1 = ["pi0_E_1", "pi0_px_1", "pi0_py_1", "pi0_pz_1"]
pi0_2 = ["pi0_E_2", "pi0_px_2", "pi0_py_2", "pi0_pz_2"]

pi2_1 = ["pi2_E_1", "pi2_px_1", "pi2_py_1", "pi2_pz_1"]
pi2_2 = ["pi2_E_2", "pi2_px_2", "pi2_py_2", "pi2_pz_2"]

pi3_1 = ["pi3_E_1", "pi3_px_1", "pi3_py_1", "pi3_pz_1"]
pi3_2 = ["pi3_E_2", "pi3_px_2", "pi3_py_2", "pi3_pz_2"]


pis = pi_1 + pi_2 + pi2_1 + pi2_2 + pi3_1 + pi3_2 

df = tree.pandas.df(variables)# + pis)
df2 = tree2.pandas.df(variables)# + pis)


#Will need improvement later

df['index'] =  np.arange(0, len(df), 1)
df2['index'] =  np.arange(0, len(df2), 1)


In [5]:
import random
random.seed(123456)


df_1 = df[
      (df["tau_decay_mode_1"] == 1) 
    & (df["tau_decay_mode_2"] == 1) 
    & (df["mva_dm_1"] == 1) 
    & (df["mva_dm_2"] == 1)
   
]



df2_1 = df2[
      (df2["tau_decay_mode_1"] == 1) 
    & (df2["tau_decay_mode_2"] == 1) 
    & (df2["mva_dm_1"] == 1) 
    & (df2["mva_dm_2"] == 1)
   
]


df_ps = df_1[
      (df_1["rand"]<df_1["wt_cp_ps"]/2)
]

df_sm = df_1[
      (df_1["rand"]<df_1["wt_cp_sm"]/2)
]


df2_ps = df2_1[
      (df2_1["rand"]<df2_1["wt_cp_ps"]/2)
]

df2_sm = df2_1[
      (df2_1["rand"]<df2_1["wt_cp_sm"]/2)
]

In [6]:
# create target labels (y)
         
# prepare the target labels
y_sm = pd.DataFrame(np.ones(df_sm.shape[0]))
y_ps = pd.DataFrame(np.zeros(df_ps.shape[0]))

y2_sm = pd.DataFrame(np.ones(df2_sm.shape[0]))
y2_ps = pd.DataFrame(np.zeros(df2_ps.shape[0]))

y = pd.concat([y_sm, y_ps,y2_sm, y2_ps])
y.columns = ["class"]

In [7]:
# prepare the dataframe to use in training
X = pd.concat([df_sm, df_ps, df2_sm, df2_ps])

# drop any other variables that aren't required in training

X2 = X.drop([
            "wt_cp_sm","wt_cp_ps","wt_cp_mm", "rand",
            "tau_decay_mode_1","tau_decay_mode_2","mva_dm_1","mva_dm_2",
            "deepTauVsJets_medium_1","deepTauVsJets_medium_2",
            "deepTauVsEle_vvloose_1","deepTauVsEle_vvloose_2",
            "deepTauVsMu_vloose_1","deepTauVsMu_vloose_2",
            "trg_doubletau",
           ], axis=1).reset_index(drop=True) 

In [8]:
X1_train,X1_test, y1_train, y1_test  = train_test_split(
    X2,
    y,
    test_size=0.2,
    random_state=123456,
    stratify=y.values,
)


In [9]:
xgb_params = {
    "objective": "binary:logistic",
    "max_depth": 5,
    "learning_rate": 0.02,
    "silent": 1,
    "n_estimators": 1000,
    "subsample": 0.9,
    "seed": 123451,
}

In [10]:

xgb_clf = xgb.XGBClassifier(**xgb_params, use_label_encoder=False)
xgb_clf.fit(
    X1_train,
    y1_train,
    early_stopping_rounds=200, # stops the training if doesn't improve after 200 iterations
    eval_set=[(X1_train, y1_train), (X1_test, y1_test)],
    eval_metric = "auc", # can use others
    verbose=True,
)

[15:32:21] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-auc:0.58900	validation_1-auc:0.58284
[1]	validation_0-auc:0.59079	validation_1-auc:0.58480
[2]	validation_0-auc:0.59236	validation_1-auc:0.58613
[3]	validation_0-auc:0.59244	validation_1-auc:0.58618
[4]	validation_0-auc:0.59296	validation_1-auc:0.58671
[5]	validation_0-auc:0.59321	validation_1-auc:0.58681
[6]	validation_0-auc:0.59308	validation_1-auc:0.58674
[7]	validation_0-auc:0.59326	validation_1-auc:0.58681
[8]	validation_0-auc:0.59329	validation_1-auc:0.58681
[9]	validation_0-auc:0.59331	validation_1-auc:0.58684
[10]	validation_0-auc:0.59351	validation_1-auc:0.58700
[11]	validation_0-auc:0.59357	validation_1-auc:0.58699
[12]	validation_0-auc:

[143]	validation_0-auc:0.60222	validation_1-auc:0.59091
[144]	validation_0-auc:0.60227	validation_1-auc:0.59091
[145]	validation_0-auc:0.60232	validation_1-auc:0.59092
[146]	validation_0-auc:0.60237	validation_1-auc:0.59094
[147]	validation_0-auc:0.60242	validation_1-auc:0.59094
[148]	validation_0-auc:0.60244	validation_1-auc:0.59094
[149]	validation_0-auc:0.60248	validation_1-auc:0.59096
[150]	validation_0-auc:0.60250	validation_1-auc:0.59094
[151]	validation_0-auc:0.60255	validation_1-auc:0.59096
[152]	validation_0-auc:0.60258	validation_1-auc:0.59096
[153]	validation_0-auc:0.60261	validation_1-auc:0.59094
[154]	validation_0-auc:0.60266	validation_1-auc:0.59093
[155]	validation_0-auc:0.60269	validation_1-auc:0.59094
[156]	validation_0-auc:0.60273	validation_1-auc:0.59093
[157]	validation_0-auc:0.60277	validation_1-auc:0.59093
[158]	validation_0-auc:0.60281	validation_1-auc:0.59092
[159]	validation_0-auc:0.60284	validation_1-auc:0.59092
[160]	validation_0-auc:0.60289	validation_1-auc:

[290]	validation_0-auc:0.60567	validation_1-auc:0.58915
[291]	validation_0-auc:0.60569	validation_1-auc:0.58914
[292]	validation_0-auc:0.60571	validation_1-auc:0.58910
[293]	validation_0-auc:0.60574	validation_1-auc:0.58909
[294]	validation_0-auc:0.60575	validation_1-auc:0.58907
[295]	validation_0-auc:0.60578	validation_1-auc:0.58905
[296]	validation_0-auc:0.60581	validation_1-auc:0.58902
[297]	validation_0-auc:0.60582	validation_1-auc:0.58901
[298]	validation_0-auc:0.60583	validation_1-auc:0.58900
[299]	validation_0-auc:0.60584	validation_1-auc:0.58899
[300]	validation_0-auc:0.60585	validation_1-auc:0.58898
[301]	validation_0-auc:0.60586	validation_1-auc:0.58897
[302]	validation_0-auc:0.60587	validation_1-auc:0.58896
[303]	validation_0-auc:0.60588	validation_1-auc:0.58895
[304]	validation_0-auc:0.60589	validation_1-auc:0.58896
[305]	validation_0-auc:0.60590	validation_1-auc:0.58895
[306]	validation_0-auc:0.60591	validation_1-auc:0.58894
[307]	validation_0-auc:0.60593	validation_1-auc:

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.02, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=8, num_parallel_tree=1,
              random_state=123451, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, seed=123451, silent=1, subsample=0.9,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [11]:
y1_proba1 = xgb_clf.predict_proba(X2)
auc = roc_auc_score(y, y1_proba1[:,1])

print(auc)

0.6002285161049092


In [13]:
#plt.savefig('Without_pis2.png')
plt.close()

In [14]:
# Add back the items that we need

X2['BDTweights'] = y1_proba1.T[0]

array_wt_cp_sm = np.array(X["wt_cp_sm"])
X2 = X2.assign(wt_cp_sm=array_wt_cp_sm)

array_wt_cp_ps = np.array(X["wt_cp_ps"])
X2 = X2.assign(wt_cp_ps=array_wt_cp_ps)

array_wt_cp_mm = np.array(X["wt_cp_mm"])
X2 = X2.assign(wt_cp_mm=array_wt_cp_mm)

array_rand = np.array(X["rand"])
X2 = X2.assign(rand=array_rand)

In [15]:
df_ps = X2[
      (X2["rand"]<X2["wt_cp_ps"]/2)
]

df_sm = X2[
      (X2["rand"]<X2["wt_cp_sm"]/2)
]

In [16]:
# plt.hist(X2['wt_cp_sm'], bins = 100, label = 'wt_cp_sm', alpha = 0.5)
# plt.hist(X2['wt_cp_ps'], bins = 100, label = 'wt_cp_ps', alpha = 0.5)
# plt.xlabel('weights')
# plt.ylabel('Occurences')
# plt.legend()
# plt.title('Checks weights')
# plt.savefig('checks_weights.png')
# plt.close()

In [17]:
plt.hist(df_ps['BDTweights'], bins = 100, alpha = 0.5, label = 'Pseudoscalar')
plt.hist(df_sm['BDTweights'], bins = 100, alpha = 0.5, label = 'Standard Model')
plt.xlabel('predict_proba')
plt.ylabel('Occurences')
plt.title('Probability of cp even for rho rho training without 4 vector\nwith reweighting')
plt.legend()
plt.savefig('Test_with_weights_fullVBFGluGlu_2.png')
plt.close()

In [18]:
#  define a function to plot the ROC curves - just makes the roc_curve look nicer than the default
def plot_roc_curve(fpr, tpr, auc, string):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, label = 'Auto-made')
    ax.set(xlabel='False Positive Rate', ylabel='True Positive Rate')
    ax.grid()
    ax.text(0.6, 0.3, 'ROC AUC Score: {:.3f}'.format(auc),
            bbox=dict(boxstyle='square,pad=0.3', fc='white', ec='k'))
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]), np.max([ax.get_xlim(), ax.get_ylim()])]
    ax.plot(lims, lims, 'k--')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    plt.savefig('%s'%string)

In [19]:
y1_proba1 = xgb_clf.predict_proba(X1_test)
plt.close()
auc = roc_auc_score(y1_test, y1_proba1[:,1])
fpr, tpr, _ = roc_curve(y1_test, y1_proba1[:,1])
plot_roc_curve(fpr, tpr, auc, 'rho_rho_GluGLu_2.png')

In [20]:
#all you are doing is computing the fraction of CP-even events with score>x [which is the true positive]
#and fraction of CP-odd events with score>x [which is the flase positive] for lots of values of x, 
#and then plotting it on a graph

def hand_made_roc(df_ps, df_sm):
    fraction_sm = []
    fraction_ps = []
    array = np.linspace(0, 1, 1000)
    for x in array:
        sup_ps = np.where(df_ps['BDTweights']>x, 0, 1)
        fraction_ps.append(np.sum(sup_ps)/len(df_ps))
        
        sup_sm = np.where(df_sm['BDTweights']>x, 0, 1)
        fraction_sm.append(np.sum(sup_sm)/len(df_sm))
    return fraction_ps, fraction_sm


In [21]:
fraction_ps, fraction_sm = hand_made_roc(df_ps, df_sm)

In [22]:

plt.plot(fraction_ps, fraction_sm, label = 'Hand-made')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
array = np.linspace(0, 1, 100)
plt.plot(array, array, 'k--')
plt.legend()
plt.savefig('Test_self_roc_5')
plt.close()

In [23]:
plt.close()

X2['BDTweights'] = y1_proba1.T[0]

#X2['BDTweights'] = y1_proba.T[1]
nbins = 100
X2["BDTweights"].plot(kind="hist", weights=X2["wt_cp_sm"], bins=nbins, histtype=u'step', lw=3,  label='CP-even - weighted', density='True')
X2["BDTweights"].plot(kind="hist", weights=X2["wt_cp_ps"], bins=nbins, histtype=u'step', lw=3,  label='CP-odd - weighted', density='True')
df_ps['BDTweights'].plot(kind = "hist", bins = nbins, histtype=u'step', lw=3, label = 'Pseudoscalar ps', density='True')
df_sm['BDTweights'].plot(kind = "hist", bins = nbins, histtype=u'step', lw=3, label = 'Standard Model sm', density='True')
leg=plt.legend()
plt.grid()
plt.title("BDT scores - densities")
plt.savefig('Weighted2_compa_5')


ValueError: Length of values (75728) does not match length of index (378638)

In [24]:
####
X2 = X2.drop([
            "wt_cp_sm","wt_cp_ps","wt_cp_mm", "rand","BDTweights"], axis=1).reset_index(drop=True) 

y2_proba = xgb_clf.predict_proba(X2)



data_prob=pd.DataFrame(y2_proba)
data_prob.columns=['odd','even']
data_prob["label"]=y.values
print (data_prob[:10])

y_even=data_prob[data_prob["label"]==1]
print(y_even[:2])


y_odd=data_prob[data_prob["label"]==0]
print(y_odd[:2])

print(y1_test.values)
print(y.values)


        odd      even  label
0  0.535305  0.464695    1.0
1  0.379426  0.620574    1.0
2  0.404715  0.595285    1.0
3  0.550619  0.449381    1.0
4  0.524303  0.475697    1.0
5  0.498268  0.501732    1.0
6  0.450412  0.549588    1.0
7  0.554860  0.445140    1.0
8  0.491298  0.508702    1.0
9  0.506300  0.493700    1.0
        odd      even  label
0  0.535305  0.464695    1.0
1  0.379426  0.620574    1.0
            odd      even  label
71338  0.637911  0.362089    0.0
71339  0.535305  0.464695    0.0
[[0.]
 [1.]
 [1.]
 ...
 [0.]
 [1.]
 [1.]]
[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]]


In [25]:

plt.close()
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
nbins=10
#density='false'
y_even["even"].hist(bins=nbins, histtype=u'step', lw=3, label='CP-even')#, density=True)
y_odd["even"].hist(bins=nbins, histtype=u'step', lw=3, label='Cp-odd')#, density=True)
leg=plt.legend(loc=2)
plt.title("BDT scores - few bins")
plt.savefig('with_correct_DATASET')



In [26]:
import pickle

with open('tt_RhoRho_Model_GluGluVBF_xgb.pkl', 'wb') as f:
        pickle.dump(xgb_clf, f, protocol=2)


In [27]:
from xgboost import __version__

In [28]:
print(xgb.__version__)



1.4.2


In [41]:

y2_proba = xgb_clf.predict_proba(X1_test)



data_prob=pd.DataFrame(y2_proba)
data_prob.columns=['odd','even']
data_prob["label"]=y1_test.values
print (data_prob[:10])




### Tests to compare with NN
def probThreshold(inpProb):
    if inpProb > 0.5:
        return(1)
    else:
        return(0)
    
data_prob["evenProb"]=data_prob["even"].apply(probThreshold)

y_even=data_prob[data_prob["label"]==1]
print(y_even[:2])


y_odd=data_prob[data_prob["label"]==0]
print(y_odd[:2])

print(y1_test.values)
print(y.values)

        odd      even  label
0  0.499957  0.500043    0.0
1  0.600596  0.399404    1.0
2  0.383897  0.616103    1.0
3  0.499301  0.500699    0.0
4  0.443098  0.556902    0.0
5  0.535801  0.464199    1.0
6  0.567748  0.432252    0.0
7  0.516473  0.483527    1.0
8  0.609656  0.390344    0.0
9  0.435761  0.564239    1.0
        odd      even  label  evenProb
1  0.600596  0.399404    1.0         0
2  0.383897  0.616103    1.0         1
        odd      even  label  evenProb
0  0.499957  0.500043    0.0         1
3  0.499301  0.500699    0.0         1
[[0.]
 [1.]
 [1.]
 ...
 [0.]
 [1.]
 [1.]]
[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]]


In [44]:
print(sum(y_even["evenProb"])/len(y_even["even"]))
print(1-sum(y_odd["evenProb"])/len(y_odd["even"]))

0.5667389694212253
0.5596192384769538


In [40]:
check = data_prob['evenProb']-data_prob['label']
check2 = np.where(check==0, 1, 0)
print(sum(check2)/len(check2))

0.563173462919924
